# Process MODIS Vegetation Indices

### Prepare Workspace

In [1]:
# Import system libraries
import os
import sys

# Import data manipulation libraries
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, mapping

# Import geospatial libraries
import geopandas as gpd
import xarray as xr
import rioxarray
import rasterio.mask

# Import API libraries
import pystac_client
import planetary_computer
import odc.stac
import rich.table

# Import visualisation libraries (optional)
import xrspatial
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation

# Set working directory
os.chdir('/Users/jessicarapson/Documents/GitHub/water-supply-forecast')

/Users/jessicarapson/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


### Load Data from API

In [178]:
# Call API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Load in site geospatial data
gdf_sites = gpd.read_file('assets/data/geospatial.gpkg')

# Initialize an empty list to store catchment bounding boxes
site_bboxes = []

# Iterate through each polygon (catchment) in the GeoDataFrame
for index, row in gdf_sites.iterrows():
    # Get the bounding box for each polygon
    bbox = row.geometry.bounds  # Extract the bounding box as (minx, miny, maxx, maxy)
    site_bboxes.append(bbox)  # Append the bounding box to the list
    
# Initialise dataframes to store extracted information
df_jan = pd.DataFrame(
    {'site_id': gdf_sites['site_id'],'year': np.nan,'month': np.nan,
     'max_veg':np.nan,'min_veg': np.nan,
     'mean_veg': np.nan,'med_veg': np.nan,'percent_no_veg': np.nan,
     'percent_bare_soil': np.nan,'percent_any_veg': np.nan,
     'percent_sparse_veg': np.nan,'percent_dense_veg': np.nan})
df_apr = df_jan.copy()
df_jul = df_jan.copy()
df_oct = df_jan.copy()

# Create dataframe of months
months_dataframes = {'January': df_jan, 'April': df_apr, 'July': df_jul, 'October': df_oct}

# Collect data for each year
df_all = []

# Loop through years
for year in range(2001,2023 + 1):

    # Loop through catchment polygons
    for i in range(0,len(gdf_sites)):
        print("Processing MODIS for:", str(year), gdf_sites.iloc[i]['site_id'],
          f"({i + 1}/{len(gdf_sites)})")

        # Load the catchment polygon
        catchment_polygon = gdf_sites.geometry.iloc[i]

        # Select catchment bounding box
        bbox = site_bboxes[i]

        # Select dates
        months = {
            "January": "01",
            "April": "04",
            "July": "07",
            "October": "10",
        }
        items_season = dict()

        # Search using bounding box coordinates
        items_full = []
        for name, number in months.items():
            datetime = f"{year}-{number}"
            search = catalog.search(
                collections=["modis-13A1-061"],
                bbox=bbox,
                datetime=datetime,
            )
            items_season[name] = search.item_collection()[0]
            items = list(search.item_collection())
            items_trunc = items[0:len(set([tuple(x.bbox) for x in items]))]
            items_full += items_trunc

        # Load and merge data into xarray
        datasets = []
        item_num = 0
        for item in items_full:
            item_num += 1
            print("Processing item:", f"{item_num}/{len(items_full)}")
            data = odc.stac.load(
                [item],
                crs="EPSG:32610",
                bands="500m_16_days_NDVI",
                resolution=500,
                bbox=bbox)
            try:
                # Clip data for each catchment polygon
                data_clipped = data.rio.clip(
                    gdf_sites.geometry.apply(mapping)[[i]], gdf_sites.crs)
                # Mask areas outside the catchment polygon with NaN
                mask = ~data_clipped.isnull()
                data_clipped = data_clipped.where(mask)
                # Append data
                datasets.append(data_clipped)
            except rioxarray.exceptions.NoDataInBounds:
                continue  # Skip to the next item if no data is found

        # Merge the datasets using xarray
        merged_data = xr.concat(datasets, dim='item_index').mean(dim='item_index')

        # Convert data to raster
        raster = items_season["January"].assets["500m_16_days_NDVI"].extra_fields["raster:bands"]
        data = merged_data["500m_16_days_NDVI"] * raster[0]["scale"]

        # Clip data again so outside values are NaN
        data_clipped = data.rio.clip(gdf_sites.geometry.apply(mapping)[[i]], gdf_sites.crs)

        # Iterate through each month DataFrame
        for month, df in months_dataframes.items():

            # Label year
            df.at[i,'year'] = year

            # Select season
            data_season = data_clipped.sel(time=data['time.month'] == int(months[month]))[0]

             # Label month
            df.at[i,'month'] = np.datetime_as_string(data_season.time.values, unit='M')[-2:]

            # Extract mean, minimum, and average vegetation for catchment
            df.at[i,'max_veg'] = np.nanmax(data_season)
            df.at[i,'min_veg'] = np.nanmin(data_season)
            df.at[i,'mean_veg'] = np.nanmean(data_season)
            df.at[i,'med_veg'] = np.nanmedian(data_season)

            # Calculate percent vegetation exceeding various thresholds
            total_cells = np.sum(~np.isnan(data_season.values))
            df.at[i,'percent_no_veg'] = np.sum(data_season.values < 0) / total_cells
            df.at[i,'percent_bare_soil'] = np.sum(
                (data_season.values < 0.01) & (data_season.values > -0.01)) / total_cells
            df.at[i,'percent_any_veg'] = np.sum(data_season.values > 0) / total_cells
            df.at[i,'percent_sparse_veg'] = np.sum(
                (data_season.values > 0.1) & (data_season.values < 0.5)) / total_cells
            df.at[i,'percent_dense_veg'] = np.sum(data_season.values > 0.6) / total_cells

             # Export clipped and compressed raster file
            data_season.rio.to_raster('assets/data/modis/' + gdf_sites.iloc[i].site_id
                                      + '_' + str(year) + '_' + month + '_nvdi.tif')

    # Combine month dataframes
    df_year = pd.concat([df_jan, df_apr, df_jul, df_oct], ignore_index=True)
    df_all.append(df_year)
    print('\n###################################################\n')

# Export dataframe
df = pd.concat(df_all, ignore_index=True)
df.to_csv('assets/data/modis/modis_summary.csv', index=False)

Processing MODIS for: 2001 hungry_horse_reservoir_inflow (1/26)
Processing item: 1/4
Processing item: 2/4
Processing item: 3/4
Processing item: 4/4


/var/folders/ws/9hc4t42j3ls9vb_fw81ddmhw0000gn/T/ipykernel_88373/4063739911.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i,'month'] = np.datetime_as_string(data_season.time.values, unit='M')[-2:]
/var/folders/ws/9hc4t42j3ls9vb_fw81ddmhw0000gn/T/ipykernel_88373/4063739911.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '04' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i,'month'] = np.datetime_as_string(data_season.time.values, unit='M')[-2:]
/var/folders/ws/9hc4t42j3ls9vb_fw81ddmhw0000gn/T/ipykernel_88373/4063739911.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '07' has dtype incompatible with float64, p

Processing MODIS for: 2001 snake_r_nr_heise (2/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2001 pueblo_reservoir_inflow (3/26)
Processing item: 1/4
Processing item: 2/4
Processing item: 3/4
Processing item: 4/4
Processing MODIS for: 2001 sweetwater_r_nr_alcova (4/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2001 missouri_r_at_toston (5/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2001 animas_r_at_durango (6/26)
Processing item: 1/4
Processing item: 2/4
Processing item: 3/4
Processing item: 4/4
Processing MODIS for: 2001 yampa_r_nr_maybell (7/26)


Processing item: 5/7
Processing item: 6/7
Processing item: 7/7
Processing MODIS for: 2002 fontenelle_reservoir_inflow (14/26)
Processing item: 1/14
Processing item: 2/14
Processing item: 3/14
Processing item: 4/14
Processing item: 5/14
Processing item: 6/14
Processing item: 7/14
Processing item: 8/14
Processing item: 9/14
Processing item: 10/14
Processing item: 11/14
Processing item: 12/14
Processing item: 13/14
Processing item: 14/14
Processing MODIS for: 2002 weber_r_nr_oakley (15/26)
Processing item: 1/7
Processing item: 2/7
Processing item: 3/7
Processing item: 4/7
Processing item: 5/7
Processing item: 6/7
Processing item: 7/7
Processing MODIS for: 2002 san_joaquin_river_millerton_reservoir (16/26)
Processing item: 1/7
Processing item: 2/7
Processing item: 3/7
Processing item: 4/7
Processing item: 5/7
Processing item: 6/7
Processing item: 7/7
Processing MODIS for: 2002 merced_river_yosemite_at_pohono_bridge (17/26)
Processing item: 1/7
Processing item: 2/7
Processing item: 3/7
Proc

Processing MODIS for: 2003 san_joaquin_river_millerton_reservoir (16/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2003 merced_river_yosemite_at_pohono_bridge (17/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2003 american_river_folsom_lake (18/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2003 colville_r_at_kettle_falls (19/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2003 stehekin_r_at_stehekin (20/26)

...

Processing item: 22/24
Processing item: 23/24
Processing item: 24/24
Processing MODIS for: 2019 libby_reservoir_inflow (8/26)
Processing item: 1/24
Processing item: 2/24
Processing item: 3/24
Processing item: 4/24
Processing item: 5/24
Processing item: 6/24
Processing item: 7/24
Processing item: 8/24
Processing item: 9/24
Processing item: 10/24
Processing item: 11/24
Processing item: 12/24
Processing item: 13/24
Processing item: 14/24
Processing item: 15/24
Processing item: 16/24
Processing item: 17/24
Processing item: 18/24
Processing item: 19/24
Processing item: 20/24
Processing item: 21/24
Processing item: 22/24
Processing item: 23/24
Processing item: 24/24
Processing MODIS for: 2019 boise_r_nr_boise (9/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2019 green_r_bl_howard_a_hanson_dam (10/26)
Processing item: 1/8
Processing item: 2/8
Proc

Processing item: 12/24
Processing item: 13/24
Processing item: 14/24
Processing item: 15/24
Processing item: 16/24
Processing item: 17/24
Processing item: 18/24
Processing item: 19/24
Processing item: 20/24
Processing item: 21/24
Processing item: 22/24
Processing item: 23/24
Processing item: 24/24
Processing MODIS for: 2020 boise_r_nr_boise (9/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2020 green_r_bl_howard_a_hanson_dam (10/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2020 taylor_park_reservoir_inflow (11/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS

Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2021 green_r_bl_howard_a_hanson_dam (10/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2021 taylor_park_reservoir_inflow (11/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2021 dillon_reservoir_inflow (12/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2021 ruedi_reservoir_inflow (13/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4

Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2022 ruedi_reservoir_inflow (13/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2022 fontenelle_reservoir_inflow (14/26)
Processing item: 1/14
Processing item: 2/14
Processing item: 3/14
Processing item: 4/14
Processing item: 5/14
Processing item: 6/14
Processing item: 7/14
Processing item: 8/14
Processing item: 9/14
Processing item: 10/14
Processing item: 11/14
Processing item: 12/14
Processing item: 13/14
Processing item: 14/14
Processing MODIS for: 2022 weber_r_nr_oakley (15/26)
Processing item: 1/7
Processing item: 2/7
Processing item: 3/7
Processing item: 4/7
Processing item: 5/7
Processing item: 6/7
Processing item: 7/7
Processing MODIS for: 2022 san

Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/6
Processing item: 5/6
Processing item: 6/6
Processing MODIS for: 2023 stehekin_r_at_stehekin (20/26)
Processing item: 1/12
Processing item: 2/12
Processing item: 3/12
Processing item: 4/12
Processing item: 5/12
Processing item: 6/12
Processing item: 7/12
Processing item: 8/12
Processing item: 9/12
Processing item: 10/12
Processing item: 11/12
Processing item: 12/12
Processing MODIS for: 2023 detroit_lake_inflow (21/26)
Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/6
Processing item: 5/6
Processing item: 6/6
Processing MODIS for: 2023 virgin_r_at_virtin (22/26)
Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/6
Processing item: 5/6
Processing item: 6/6
Processing MODIS for: 2023 skagit_ross_reservoir (23/26)
Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/6
Processing item: 5/6
Processing item: 6/6
Proce